**PRE-PROCESSING: REINHARD NORMALIZATION**

In [ ]:
# INSTALLINGS

!pip install imagecodecs
!pip install histomicstk --find-links https://girder.github.io/large_image_wheels
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.1.2.30
!pip install plotly==5.3.1

!pip install pyyaml==5.4.1

     |████████████████████████████████| 31.0 MB 1.5 MB/s 
Looking in links: https://girder.github.io/large_image_wheels
     |████████████████████████████████| 524 kB 10.6 MB/s 
     |████████████████████████████████| 4.7 MB 10.6 MB/s 
     |████████████████████████████████| 57 kB 4.9 MB/s 
     |████████████████████████████████| 165 kB 52.1 MB/s 
     |████████████████████████████████| 47.7 MB 2.1 MB/s 
     |████████████████████████████████| 19.4 MB 11.4 MB/s 
     |████████████████████████████████| 133 kB 47.9 MB/s 
     |████████████████████████████████| 44 kB 2.6 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 41.4 MB 1.2 MB/s 
     |████████████████████████████████| 65.3 MB 54 kB/s 
     |████████████████████████████████| 829 kB 50.0 MB/s 
     |████████████████████████████████| 132 kB 28.3 MB/s 
     |████████████████████████████████| 8.5 MB 41.0 MB/s 
     |██████████

In [ ]:
# LINKAGE TO GOOGLE DRIVE AND LIBRERIES IMPORTING

from google.colab import drive
drive.mount('/content/drive')

import os
import random
import numpy as np
import plotly.express as px
import imagecodecs
import cv2

from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage.io import imread, imshow, imsave
from skimage.transform import resize
from skimage.segmentation import mark_boundaries
from scipy import ndimage
from skimage.util import img_as_float,img_as_ubyte, crop
from skimage import exposure
from skimage.morphology import binary_dilation
from keras.utils.np_utils import to_categorical
from skimage import measure, morphology
from scipy.ndimage import median_filter

from skimage.color import rgb2hsv
from __future__ import print_function
import histomicstk as htk

In [ ]:
# DATASET UNRAR: LOADING DATASET IN COLAB

!pip install unrar
!unrar x "drive/MyDrive/cytology challenge condivisa/00_DATASET/train.rar"     # unraring training set
!unrar x "drive/MyDrive/cytology challenge condivisa/00_DATASET/validation.rar"   # unraring validation set
!unrar x "drive/MyDrive/cytology challenge condivisa/00_DATASET/test.rar"         # unraring test set


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from drive/MyDrive/cytology challenge condivisa/00_DATASET/train.rar

Creating    train                                                     OK
Creating    train/images                                              OK
Extracting  train/images/102.tif                                           0%  OK 
Extracting  train/images/104.tif                                           0%  1%  OK 
Extracting  train/images/106.tif                                           1%  OK 
Extracting  train/images/108.tif                                           1%  2%  OK 
Extracting  train/images/109.tif                                           2%  OK 
Extracting  train/images/111.tif                                           2%  3%  OK 
Extracting  train/images/112.tif                                           3%  OK 
Extracting  train/images/114.tif    

In [ ]:
# SETTINGS OF CURRENT PRE-PROCESSING

pre_proc_name = 'NORM_IL'  # name of current pre-processing <------------------------------- CHANGE HERE
rsz = 512  # resizing size (resize images to rsz x rsz) <-------------------------------- CHANGE HERE
NUM_CLASSES = 3 # number of classes choosen for the problem <---------------------------- CHANGE HERE

In [ ]:
# LOAD RESIZED TRAINING SET AND VALIDATION SET

# load
images = np.load('drive/MyDrive/cytology challenge condivisa/01_PRE-PROCESSED/IL1.npz')  # images loading <--------------------------------------- CHANGE HERE
labels = np.load('drive/MyDrive/cytology challenge condivisa/01_PRE-PROCESSED/IL1_manual_mask_512x512.npz')  # manual annotations loading <-------- CHANGE HERE
X_tr = images['X_tr']
X_vl = images['X_vl']
Y_tr = labels['Y_tr']
Y_vl = labels['Y_vl']
images.close()
labels.close()
del images,labels

In [ ]:
# REIHNARD NORMALIZATION


fig = px.imshow(X_tr[1:10], animation_frame=0, binary_string=True, labels=dict(x="normalized images - VLS",animation_frame="slice"))
fig.show()
img_ref = X_tr[1]


# fig = px.imshow(img_ref)
# fig.show()

mean_ref, std_ref = htk.preprocessing.color_conversion.lab_mean_std(img_ref)

for n in tqdm(range(len(X_tr)), total=len(X_tr)):

    # loading
    img = np.copy(X_tr[n])
    img_norm = htk.preprocessing.color_normalization.reinhard(img, mean_ref, std_ref)
    X_tr[n] = np.copy(img_norm)


fig = px.imshow(X_tr[1:30], animation_frame=0, binary_string=True, labels=dict(x="normalized images - TRS",animation_frame="slice"))
fig.show()

img = []
img_norm = []

for n in tqdm(range(len(X_vl)), total=len(X_vl)):

    # loading
    img = np.copy(X_vl[n])
    img_norm = htk.preprocessing.color_normalization.reinhard(img, mean_ref, std_ref)
    X_vl[n] = np.copy(img_norm)

fig = px.imshow(X_vl[1:10], animation_frame=0, binary_string=True, labels=dict(x="normalized images - VLS",animation_frame="slice"))
fig.show()


In [ ]:
# SAVING PRE-PROCESSED

np.savez(os.path.join('drive/MyDrive/cytology challenge condivisa/01_PRE-PROCESSED',pre_proc_name),X_tr=X_tr,X_vl=X_vl)
#pre_proc_annotations_name = 'PRA_manual_mask_' + str(rsz) + 'x' + str(rsz)  # <------------------------------------------------- comment this line if preprocessed annotations of wanted size are already existing
#np.savez(os.path.join('drive/MyDrive/cytology challenge/01_PRE-PROCESSED',pre_proc_annotations_name),Y_tr=Y_tr,Y_vl=Y_vl)  # <-- comment this line if preprocessed annotations of wanted size are already existing